In [1]:
#
# _____ ______  _______      _  _  _ _______ _______ _______  _____  __   _      _     _
#   |   |_____] |  |  |      |  |  | |_____|    |    |______ |     | | \  |       \___/ 
# __|__ |_____] |  |  |      |__|__| |     |    |    ______| |_____| |  \_|      _/   \_                                                                                       
#

In [2]:
!pip install "tracing_auto_instrumentation[ibm]"

In [3]:
import dotenv
import logging
import os

from enum import Enum

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

from tracing_auto_instrumentation.ibm import wrap_watson

In [4]:
logger = logging.getLogger(__name__)

In [5]:
class Mode(Enum):
    GENERATE = "GENERATE"
    GENERATE_TEXT = "GENERATE_TEXT"


def init_watson_model() -> Model:
    # To display example params enter
    GenParams().get_example_values()
    generate_params = {GenParams.MAX_NEW_TOKENS: 25}

    watson_model = Model(
        model_id=ModelTypes.GRANITE_13B_CHAT_V2,
        params=generate_params,
        credentials=dict(
            api_key=os.getenv("WATSONX_API_KEY"),
            url="https://us-south.ml.cloud.ibm.com",
        ),
        space_id=os.getenv("WATSONX_SPACE_ID"),
        verify=None,
        validate=True,
    )

    return watson_model

In [6]:
TRACE_NAME = "elementary-my-dear-watson"

tracer: LastMileTracer = get_lastmile_tracer(
    TRACE_NAME, os.getenv("LASTMILE_API_TOKEN")
)
watson_model: Model = init_watson_model()
wrapper = wrap_watson(watson_model, tracer)

[INFO] 2024-06-17 01:14:18,659 client.py:458: Client successfully initialized
[INFO] 2024-06-17 01:14:19,671 wml_resource.py:93: Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-05-10&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&limit=200'
[DEBUG] 2024-06-17 01:14:19,672 wml_resource.py:98: Response(GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-05-10&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&limit=200): {"total_count":25,"limit":200,"first":{"href":"https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-05-10&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&limit=200"},"resources":[{"model_id":"baai/bge-large-en-v1","label":"bge-large-en-v1","provider":"baai","source":"baai","functions":[{"id":"embedding"}],"short_description":"An embedding model with version 1.5. It has 335 million parameters and an embedding dimension of 1024.","long_desc

In [7]:
def run_generation(mode: Mode, prompt: str) -> None:
    logger.info("IBM WatsonX Generation Script Starting...")

    # n.b. required for multiple api keys, make sure `.env` is in your local path
    dotenv.load_dotenv()
    
    response: str = ""
    if Mode.GENERATE == mode:
        logger.info(f"running with mode: {mode}")
        response = wrapper.generate(prompt)
    elif Mode.GENERATE_TEXT == mode:
        logger.info(f"running with mode: {mode}")
        response = wrapper.generate_text(prompt)
    else:
        logger.error(f"unsupported mode: {mode}")
    logger.info(f"response: {response}")

In [8]:
run_generation(
    mode=Mode.GENERATE,
    prompt="Can you please tell me an amusing anecdote about Thomas Edison?",
)

[INFO] 2024-06-17 01:14:20,000 1619447352.py:2: IBM WatsonX Generation Script Starting...
[INFO] 2024-06-17 01:14:20,001 1619447352.py:9: running with mode: Mode.GENERATE
[INFO] 2024-06-17 01:14:21,687 wml_resource.py:93: Successfully finished generate for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10'
[DEBUG] 2024-06-17 01:14:21,687 wml_resource.py:98: Response(POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-05-10): {"model_id":"ibm/granite-13b-chat-v2","model_version":"2.1.0","created_at":"2024-06-17T05:14:21.568Z","results":[{"generated_text":"\n\nCertainly! One of Edison's most famous inventions was the phonograph, the world's first practical device for recording and","generated_token_count":25,"input_token_count":13,"stop_reason":"max_tokens"}]}
[INFO] 2024-06-17 01:14:21,964 1619447352.py:16: response: {'model_id': 'ibm/granite-13b-chat-v2', 'model_version': '2.1.0', 'created_at': '2024-06-17T05:14:21.568Z', 'results': 